In [55]:
import pandas as pd 
import geopandas as gpd
import csv
from pathlib import Path

In [ ]:
pd.set_option("min_rows")

In [5]:
old = pd.read_csv('processed_data.csv')
old.head()

,state,district,favored,confidence,rep_nominee,dem_nominee,incumbent,third_party_cand,third_party,redistricting_voter_power,anti_gerrymandering_party,geoid,chamber,district_link,dem_cand_link,rep_cand_link,third_cand_link
0,TX,TX-HD-112,FALSE,Toss-Up,Angie Chen Button,Brandy Chambers,R,Shane Newsom,L,100.000000,D,48112,HD,https://ballotpedia.org/Texas_House_of_Represe...,https://brandykchambers.com/,http://www.angiebutton.com/,https://www.newsomforstaterep.com/
1,TX,TX-HD-26,R,Tilt,Jacey Jetton,L. Sarah DeMerchant,Open,NaN,NaN,93.418871,D,48026,HD,https://ballotpedia.org/Texas_House_of_Represe...,https://www.democratdemerchant.com/,https://www.jaceyjetton.com/,NaN
2,TX,TX-HD-66,FALSE,Toss-Up,Matt Shaheen,Sharon Hirsch,R,Shawn Jones,L,91.250686,D,48066,HD,https://ballotpedia.org/Texas_House_of_Represe...,https://www.sharon4tx.com/,http://mattshaheen.com/index.html,https://ballotpedia.org/Shawn_Jones
3,TX,TX-HD-67,FALSE,Toss-Up,Jeff Leach,Lorenzo Sanchez,R,NaN,NaN,87.276685,D,48067,HD,https://ballotpedia.org/Texas_House_of_Represe...,https://www.lorenzofortexas.com/,http://www.jeffleach.com/,NaN
4,TX,TX-HD-96,FALSE,Toss-Up,David Cook,Joe Drago,Open,Nelson Range,L,86.885977,D,48096,HD,https://ballotpedia.org/Texas_House_of_Represe...,https://dragofortexas.com/home-1,https://www.davidcookfortexas.com/,https://nelsonrangefortexas96.com


In [8]:
sheet = pd.read_csv("all_results_10_10_2020-sorted.csv")
sheet.head()

,state,district,incumbent,favored,confidence,nom_R,nom_D,nom_I,cvap,VOTER_POWER,redistricting_voter_power,r_v_p scaled,r_v_p scaled + absolute
0,TX,TX-HD-112,R,FALSE,Toss-Up,Angie Chen Button,Brandy Chambers,FALSE,109960,0.000006,0.000213,100.000000,100.000000
1,TX,TX-HD-26,FALSE,FALSE,Toss-Up,Jacey Jetton,Sarah DeMarchant,FALSE,115745,0.000005,0.000200,93.552609,93.552609
2,TX,TX-HD-92,FALSE,FALSE,Toss-Up,Jeff Cason,Jeff Whitfield,FALSE,121695,0.000005,0.000191,89.656153,89.656153
3,TX,TX-HD-67,R,FALSE,Toss-Up,Jeff Leach,Lorenzo Sanchez,FALSE,128105,0.000005,0.000184,86.154360,86.154360
4,TX,TX-HD-96,FALSE,FALSE,Toss-Up,David Cook,Joe Drago,FALSE,127240,0.000005,0.000183,85.559309,85.559309


In [24]:
# sheet['anti_gerrymandering_party'] = sheet.apply(lambda row: sheet['r_v_p scaled'])
def calc_anti_gerrymandering_party(row):
    if row['r_v_p scaled'] > 0:
        return 'D'
    if row['r_v_p scaled'] < 0:
        return 'R'

sheet['anti_gerrymandering_party'] = sheet.apply(lambda row: calc_anti_gerrymandering_party(row), axis=1)

In [20]:
# rename(columns = {'test':'TEST'}, inplace = True) 
sheet.rename(columns = {'nom_R':'rep_nominee'}, inplace = True)

In [22]:
sheet.rename(columns = {'nom_D':'dem_nominee'}, inplace = True)

In [26]:
def calc_third_party_cand(row):
    if row['nom_I']:
        return row['nom_I']
    else:
        return ''

sheet['third_party_cand'] = sheet.apply(lambda row: calc_third_party_cand(row), axis=1)

In [29]:
sheet.head()

,state,district,incumbent,favored,confidence,rep_nominee,dem_nominee,nom_I,cvap,VOTER_POWER,redistricting_voter_power,r_v_p scaled,r_v_p scaled + absolute,anti_gerrymandering_party,third_party_cand
0,TX,TX-HD-112,R,FALSE,Toss-Up,Angie Chen Button,Brandy Chambers,FALSE,109960,0.000006,0.000213,100.000000,100.000000,D,FALSE
1,TX,TX-HD-26,FALSE,FALSE,Toss-Up,Jacey Jetton,Sarah DeMarchant,FALSE,115745,0.000005,0.000200,93.552609,93.552609,D,FALSE
2,TX,TX-HD-92,FALSE,FALSE,Toss-Up,Jeff Cason,Jeff Whitfield,FALSE,121695,0.000005,0.000191,89.656153,89.656153,D,FALSE
3,TX,TX-HD-67,R,FALSE,Toss-Up,Jeff Leach,Lorenzo Sanchez,FALSE,128105,0.000005,0.000184,86.154360,86.154360,D,FALSE
4,TX,TX-HD-96,FALSE,FALSE,Toss-Up,David Cook,Joe Drago,FALSE,127240,0.000005,0.000183,85.559309,85.559309,D,FALSE


In [49]:
# converts district code from the money ball csv to a GEOID
# ex. "CT-HD-59" --> '9059'
# ex. "MN-HD-13A" --> '2713A'
def getGEOID(row, leading_zero = False):
    district_str = row['district']
    state, chamber, dist_num = district_str.split('-')
    GEOID = str(get_state_code(state)) 
    if leading_zero and len(GEOID) < 2: 
        GEOID= '0' + GEOID
    while len(dist_num) <3: 
        dist_num = '0' + dist_num
    GEOID =  GEOID + dist_num
    return GEOID

# Util method to come between state formats 
# returnType must be either 'full' 'two_digit' or 'FIPS'
def get_state_code(input, returnType = 'FIPS'):
    if (returnType != 'full' and returnType != 'two_digit' and returnType != 'FIPS'):
        raise ValueError("returnType must be either 'full' 'two_digit' or 'FIPS'")

    code_hash = {
        'AL': {'full': 'Alabama', 'two_digit': 'AL', 'FIPS': 1},
        'AK': {'full': 'Alaska', 'two_digit': 'AK', 'FIPS': 2},
        'AZ': {'full': 'Arizona', 'two_digit': 'AZ', 'FIPS': 4},
        'AR': {'full': 'Arkansas', 'two_digit': 'AR', 'FIPS': 5},
        'CA': {'full': 'California', 'two_digit': 'CA', 'FIPS': 6},
        'CO': {'full': 'Colorado', 'two_digit': 'CO', 'FIPS': 8},
        'CT': {'full': 'Connecticut', 'two_digit': 'CT', 'FIPS': 9},
        'DE': {'full': 'Delaware', 'two_digit': 'DE', 'FIPS': 10},
        'DC': {'full': 'District of Columbia', 'two_digit': 'DC', 'FIPS': 11},
        'FL': {'full': 'Florida', 'two_digit': 'FL', 'FIPS': 12},
        'GA': {'full': 'Georgia', 'two_digit': 'GA', 'FIPS': 13},
        'HI': {'full': 'Hawaii', 'two_digit': 'HI', 'FIPS': 15},
        'ID': {'full': 'Idaho', 'two_digit': 'ID', 'FIPS': 16},
        'IL': {'full': 'Illinois', 'two_digit': 'IL', 'FIPS': 17},
        'IN': {'full': 'Indiana', 'two_digit': 'IN', 'FIPS': 18},
        'IA': {'full': 'Iowa', 'two_digit': 'IA', 'FIPS': 19},
        'KS': {'full': 'Kansas', 'two_digit': 'KS', 'FIPS': 20},
        'KY': {'full': 'Kentucky', 'two_digit': 'KY', 'FIPS': 21},
        'LA': {'full': 'Louisiana', 'two_digit': 'LA', 'FIPS': 22},
        'ME': {'full': 'Maine', 'two_digit': 'ME', 'FIPS': 23},
        'MD': {'full': 'Maryland', 'two_digit': 'MD', 'FIPS': 24},
        'MA': {'full': 'Massachusetts', 'two_digit': 'MA', 'FIPS': 25},
        'MI': {'full': 'Michigan', 'two_digit': 'MI', 'FIPS': 26},
        'MN': {'full': 'Minnesota', 'two_digit': 'MN', 'FIPS': 27},
        'MS': {'full': 'Mississippi', 'two_digit': 'MS', 'FIPS': 28},
        'MO': {'full': 'Missouri', 'two_digit': 'MO', 'FIPS': 29},
        'MT': {'full': 'Montana', 'two_digit': 'MT', 'FIPS': 30},
        'NE': {'full': 'Nebraska', 'two_digit': 'NE', 'FIPS': 31},
        'NV': {'full': 'Nevada', 'two_digit': 'NV', 'FIPS': 32},
        'NH': {'full': 'New Hampshire', 'two_digit': 'NH', 'FIPS': 33},
        'NJ': {'full': 'New Jersey', 'two_digit': 'NJ', 'FIPS': 34},
        'NM': {'full': 'New Mexico', 'two_digit': 'NM', 'FIPS': 35},
        'NY': {'full': 'New York', 'two_digit': 'NY', 'FIPS': 36},
        'NC': {'full': 'North Carolina', 'two_digit': 'NC', 'FIPS': 37},
        'ND': {'full': 'North Dakota', 'two_digit': 'ND', 'FIPS': 38},
        'OH': {'full': 'Ohio', 'two_digit': 'OH', 'FIPS': 39},
        'OK': {'full': 'Oklahoma', 'two_digit': 'OK', 'FIPS': 40},
        'OR': {'full': 'Oregon', 'two_digit': 'OR', 'FIPS': 41},
        'PA': {'full': 'Pennsylvania', 'two_digit': 'PA', 'FIPS': 42},
        'RI': {'full': 'Rhode Island', 'two_digit': 'RI', 'FIPS': 44},
        'SC': {'full': 'South Carolina', 'two_digit': 'SC', 'FIPS': 45},
        'SD': {'full': 'South Dakota', 'two_digit': 'SD', 'FIPS': 46},
        'TN': {'full': 'Tennessee', 'two_digit': 'TN', 'FIPS': 47},
        'TX': {'full': 'Texas', 'two_digit': 'TX', 'FIPS': 48},
        'UT': {'full': 'Utah', 'two_digit': 'UT', 'FIPS': 49},
        'VT': {'full': 'Vermont', 'two_digit': 'VT', 'FIPS': 50},
        'VA': {'full': 'Virginia', 'two_digit': 'VA', 'FIPS': 51},
        'WA': {'full': 'Washington', 'two_digit': 'WA', 'FIPS': 53},
        'WV': {'full': 'West Virginia', 'two_digit': 'WV', 'FIPS': 54},
        'WI': {'full': 'Wisconsin', 'two_digit': 'WI', 'FIPS': 55},
        'WY': {'full': 'Wyoming', 'two_digit': 'WY', 'FIPS': 56},
        'PR': {'full': 'Puerto Rico', 'two_digit': 'PR', 'FIPS': 72}
    }
    for row in code_hash:
        for value in code_hash[row]:
            if code_hash[row][value] == input:
                return code_hash[row][returnType]
    return None

# extracts the chamber type from the district code in the money ball csv
# ex. "CT-HD-59" --> 'HD'
def getChamber(district_str):
    state, chamber, dist_num = district_str.split('-')
    return chamber

def getChamberRow(row):
    return getChamber(row['district'])

# extracts the district name.  Used for the non-numerical Massachussets 
# districts in place of GEOID matching
def getName(district_str):
    state, chamber, dist_str = district_str.split('-')
    if len(dist_str) < 4 : return ''
    return dist_str

In [46]:
new_sheet = pd.read_csv("all_results_10_10_2020-sorted-updated.csv")


In [48]:
new_sheet['geoid'] = new_sheet.apply(lambda row: getGEOID(row), axis=1)

In [51]:
new_sheet['chamber'] = new_sheet.apply(lambda row:getChamberRow(row), axis=1)

In [63]:
url_hash = pd.read_json('url_hash.json', lines=True, orient=str)

In [121]:
# new_sheet["dem_cand_link"]
# new_sheet["rep_cand_link"]
# new_sheet["third_cand_link"]
def get_district_link(row):
    if row['district'] not in url_hash:
        return ''
    else:
        return url_hash[row['district']]

new_sheet["district_link"] = new_sheet.apply(lambda row:get_district_link(row), axis=1)

def get_rep_nom_link(row):
    if row['rep_nominee'] not in url_hash:
        return ''
    else:
        return url_hash[row['rep_nominee']]

new_sheet["rep_cand_link"] = new_sheet.apply(lambda row:get_rep_nom_link(row), axis=1)

def get_dem_nom_link(row):
    if row['dem_nominee'] not in url_hash:
        return ''
    else:
        return url_hash[row['dem_nominee']]

new_sheet["dem_cand_link"] = new_sheet.apply(lambda row:get_dem_nom_link(row), axis=1)

def get_third_nom_link(row):
    if row['third_party_cand'] == 'FALSE':
        return ''
    noms = row['third_party_cand'].split(",")
    links = []
    for nom in noms:
        if nom not in url_hash:
            pass
        else:
            links.append(url_hash[nom])
    return links

new_sheet["third_cand_link"] = new_sheet.apply(lambda row:get_third_nom_link(row), axis=1)

In [122]:
new_sheet.to_csv("all_results_10_10_2020-sorted-updated.csv")